<a href="https://colab.research.google.com/github/M-McKegney/ET5003_SEM1_2021-2/blob/MMcKegneyBranch/ET5003_Etivity2_MichaelMcKegney_20149743.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://drive.google.com/uc?export=view&id=1vK33e_EqaHgBHcbRV_m38hx6IkG0blK_" width="350"/>
</div> 

#**Artificial Intelligence - MSc**
##ET5003 - MACHINE LEARNING APPLICATIONS 

###Instructor: Enrique Naredo
###ET5003_Etivity-2

In [517]:
#@title Current Date
Today = '2021-10-01' #@param {type:"date"}


In [518]:
#@markdown ---
#@markdown ### Enter your details here:
Student_ID = "20149743" #@param {type:"string"}
Student_full_name = "Michael McKegney" #@param {type:"string"}
#@markdown ---

In [519]:
#@title Notebook information
Notebook_type = 'Example' #@param ["Example", "Lab", "Practice", "Etivity", "Assignment", "Exam"]
Version = 'Draft' #@param ["Draft", "Final"] {type:"raw"}
Submission = False #@param {type:"boolean"}

# INTRODUCTION

**Piecewise regression**, extract from [Wikipedia](https://en.wikipedia.org/wiki/Segmented_regression):

Segmented regression, also known as piecewise regression or broken-stick regression, is a method in regression analysis in which the independent variable is partitioned into intervals and a separate line segment is fit to each interval. 

* Segmented regression analysis can also be performed on 
multivariate data by partitioning the various independent variables. 
* Segmented regression is useful when the independent variables, clustered into different groups, exhibit different relationships between the variables in these regions. 

* The boundaries between the segments are breakpoints.

* Segmented linear regression is segmented regression whereby the relations in the intervals are obtained by linear regression. 

***The goal is to use advanced Machine Learning methods to predict House price.***

## Imports

In [520]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [521]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import arviz as az
from sklearn.preprocessing import StandardScaler

In [522]:
# to plot
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D

# to generate classification, regression and clustering datasets
import sklearn.datasets as dt

# to create data frames
from pandas import DataFrame

# to generate data from an existing dataset
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [523]:
# Define the seed so that results can be reproduced
seed = 11
rand_state = 11

# Define the color maps for plots
color_map = plt.cm.get_cmap('RdYlBu')
color_map_discrete = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","cyan","magenta","blue"])

# DATASET

Extract from this [paper](https://ieeexplore.ieee.org/document/9300074):

* House prices are a significant impression of the economy, and its value ranges are of great concerns for the clients and property dealers. 

* Housing price escalate every year that eventually reinforced the need of strategy or technique that could predict house prices in future. 

* There are certain factors that influence house prices including physical conditions, locations, number of bedrooms and others.


1. [Download the dataset](https://github.com/UL-ET5003/ET5003_SEM1_2021-2/tree/main/Week-3). 

2. Upload the dataset into your folder.



The challenge is to predict the final price of each house.

## Read in the dataset

**Be careful with the 'Path' location shown below i.e. make sure it is pointing to where you store the data for Etivity 2!**

The following code is being used to determine if a Jupyter Notebook or Google Colab environment is being used to load the dataset.

The development work for this Etivity was done in Google Colab.

There are 3 files to account for:
*   house_train.csv
*   house_test.csv
*   true_price.csv

In [524]:
# Checking if you are using Google Colab.

RunningInCOLAB = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # paste the path here, keep quotation marks and the trailing slash
    Path = '/content/drive/MyDrive/UL_Colab_Files/ET5003/Etivity2/'
    
else:
    
    Path = './Etivity2/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read in each ".csv" file into a Pandas dataframe and print the shape of the data.

In [525]:
# Training Dataset: 
training_file = Path + 'house_train.csv'
dftrain = pd.read_csv(training_file)
print('House train data shape: ', dftrain.shape)

# Test Dataset:
testing_file = Path + 'house_test.csv'
dftest = pd.read_csv(testing_file)
print('House test data shape: ', dftest.shape)

# Cost Dataset:
cost_file = Path + 'true_price.csv'
dfcost = pd.read_csv(cost_file)
print('True price data shape: ', dfcost.shape)


House train data shape:  (2982, 17)
House test data shape:  (500, 16)
True price data shape:  (500, 2)


Check the initial rows and review some basic statistics for each dataframe.

## Training & Test Data

**Not using this for now...**

In [526]:
# # split data into training and test
# from sklearn.model_selection import train_test_split

# # training: 70% (0.7), test: 30% (0.3) 
# # you could try any other combination 
# # but consider 50% of training as the low boundary
# X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3)

### Train dataset

In [527]:
# Show first data frame rows.
dftrain.head()

,ad_id,area,bathrooms,beds,ber_classification,county,description_block,environment,facility,features,latitude,longitude,no_of_units,price,property_category,property_type,surface
0,996887,Portmarnock,NaN,NaN,NaN,Dublin,A SELECTION OF 4 AND 5 BEDROOM FAMILY HOMES LO...,prod,NaN,None,53.418216,-6.149329,18.0,NaN,new_development_parent,NaN,NaN
1,999327,Lucan,NaN,NaN,NaN,Dublin,**Last 2 remaining houses for sale ***\n\nOn v...,prod,NaN,None,53.364917,-6.454935,3.0,NaN,new_development_parent,NaN,NaN
2,999559,Rathfarnham,NaN,NaN,NaN,Dublin,Final 4 &amp; 5 Bedroom Homes for Sale\n\nOn V...,prod,NaN,None,53.273447,-6.313821,3.0,NaN,new_development_parent,NaN,NaN
3,9102986,Balbriggan,NaN,NaN,NaN,Dublin,"Glenveagh Taylor Hill, Balbriggan\n\r\n*Ideal ...",prod,NaN,None,53.608167,-6.210914,30.0,NaN,new_development_parent,NaN,NaN
4,9106028,Foxrock,NaN,NaN,NaN,Dublin,*New phase launching this weekend Sat &amp; Su...,prod,NaN,None,53.262531,-6.181527,8.0,NaN,new_development_parent,NaN,NaN


In [528]:
# Generate descriptive statistics.
dftrain.describe()

,ad_id,bathrooms,beds,latitude,longitude,no_of_units,price,surface
count,2.982000e+03,2931.000000,2931.000000,2982.000000,2982.000000,59.000000,2.892000e+03,2431.000000
mean,1.224065e+07,1.998635,2.979188,53.355991,-6.257175,7.440678,5.323536e+05,318.851787
std,5.793037e+05,1.291875,1.468408,0.086748,0.141906,8.937081,5.678148e+05,4389.423136
min,9.968870e+05,0.000000,0.000000,51.458439,-6.521183,0.000000,1.999500e+04,3.400000
25%,1.226813e+07,1.000000,2.000000,53.298929,-6.314064,2.000000,2.800000e+05,74.100000
50%,1.237758e+07,2.000000,3.000000,53.345497,-6.252254,3.000000,3.800000e+05,100.000000
75%,1.240294e+07,3.000000,4.000000,53.388845,-6.196049,8.000000,5.750000e+05,142.000000
max,1.242836e+07,18.000000,27.000000,53.630588,-1.744995,36.000000,9.995000e+06,182108.539008


Assess count of each feature alongside the data type i.e. whether it is numeric or categorical.

In [529]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982 entries, 0 to 2981
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ad_id               2982 non-null   int64  
 1   area                2982 non-null   object 
 2   bathrooms           2931 non-null   float64
 3   beds                2931 non-null   float64
 4   ber_classification  2305 non-null   object 
 5   county              2982 non-null   object 
 6   description_block   2982 non-null   object 
 7   environment         2982 non-null   object 
 8   facility            965 non-null    object 
 9   features            2982 non-null   object 
 10  latitude            2982 non-null   float64
 11  longitude           2982 non-null   float64
 12  no_of_units         59 non-null     float64
 13  price               2892 non-null   float64
 14  property_category   2982 non-null   object 
 15  property_type       2931 non-null   object 
 16  surfac

Check each column in the dataframe how many null/Nan instances there are.

In [530]:
dftrain.isna().sum()

ad_id                    0
area                     0
bathrooms               51
beds                    51
ber_classification     677
county                   0
description_block        0
environment              0
facility              2017
features                 0
latitude                 0
longitude                0
no_of_units           2923
price                   90
property_category        0
property_type           51
surface                551
dtype: int64

There is 2982 rows and 17 columns in the training data. 
*   '**facility**' has 2017 NaN instances which means there is nearly two thirds of values missing. 
*   '**no_of_units**' has 2923 NaN instances which means there is nearly all values missing.
*   '**ad_id**' has 2982 unique values which equates to the total amount of rows so it is not offering any helpful information.
*   '**description_block**' and '**features**' have sentences/statements with inconsistent descriptions about the properties so these are not helpful for training a model. 

These features will be dropped as there is not much value being added in terms of training a model with them.




In [531]:
dftrain_adj = dftrain.drop(columns=['ad_id', 'facility','no_of_units', 'description_block', 'features'])

dftrain_adj.shape

(2982, 12)

Check how much variation there is in the features '**county**', '**environment**' and '**property_category**'.

In [532]:
dftrain_adj['county'].value_counts()

Dublin    2982
Name: county, dtype: int64

In [533]:
dftrain_adj['environment'].value_counts()

prod    2982
Name: environment, dtype: int64

In [534]:
dftrain_adj['property_category'].value_counts()

sale                      2923
new_development_parent      59
Name: property_category, dtype: int64

As '**county**' has only one value 'Dublin' and '**environment**' has only one value 'prod', these features can be removed as they are not providing any useful information. 

Only 59 out of 2982 instances (~2%) of '**property_category**' are not 'sale' so this feature is not adding a lot of information either and can be removed too.

In [535]:
dftrain_adj.drop(columns=['county', 'environment', 'property_category'], inplace=True)

In [536]:
dftrain_adj.shape

(2982, 9)

Drop all rows with NaN present as the sample/observation is incomplete and may be misleading.

In [537]:
dftrain_adj.dropna(inplace=True)

dftrain_adj.shape

(2002, 9)

Remove samples where there are 0 bathrooms and 0 bedrooms.

In [538]:
dftrain_adj.drop(dftrain_adj[dftrain_adj.beds == 0].index, inplace=True)

dftrain_adj.drop(dftrain_adj[dftrain_adj.bathrooms == 0].index, inplace=True)

dftrain_adj.shape

(1989, 9)

Check what features remain.

In [539]:
dftrain_adj.head()

,area,bathrooms,beds,ber_classification,latitude,longitude,price,property_type,surface
15,Clonee,3.0,5.0,A3,53.400454,-6.445730,935000.0,detached,321.0
26,Clondalkin,4.0,4.0,A3,53.316410,-6.385214,485000.0,semi-detached,144.0
27,Clonee,3.0,5.0,A3,53.401414,-6.446634,935000.0,detached,321.0
35,Howth,5.0,5.0,A2,53.375377,-6.056749,1475000.0,detached,312.0
38,Castleknock,2.0,2.0,A3,53.372130,-6.338466,410000.0,apartment,83.0


In [540]:
dftrain_adj['area'].value_counts()

Rathfarnham    73
Castleknock    67
Malahide       57
Lucan          55
Blackrock      51
               ..
Kilmore         1
Kiltipper       1
Greenhills      1
Kingswood       1
Garristown      1
Name: area, Length: 144, dtype: int64

In [541]:
temporary_series = dftrain_adj['area'].value_counts()

temporary_series.values

array([73, 67, 57, 55, 51, 45, 41, 39, 39, 38, 37, 37, 36, 36, 34, 33, 33,
       33, 32, 30, 29, 29, 26, 26, 25, 24, 24, 24, 24, 22, 22, 21, 20, 20,
       19, 18, 18, 17, 17, 16, 16, 16, 16, 16, 15, 15, 14, 13, 13, 13, 13,
       13, 13, 13, 13, 12, 12, 12, 12, 12, 11, 11, 11, 11, 10, 10, 10, 10,
       10,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,
        7,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1])

In [542]:
dftrain_adj['ber_classification'].value_counts()

D1                     255
D2                     241
C3                     224
C2                     210
C1                     182
E1                     154
E2                     148
G                      144
F                      135
B3                     106
B2                      69
A3                      61
SINo666of2006exempt     23
A2                      22
B1                      14
A1                       1
Name: ber_classification, dtype: int64

In [543]:
dftrain_adj['property_type'].value_counts()

semi-detached     554
apartment         530
terraced          339
detached          297
end-of-terrace    153
bungalow           56
duplex             37
townhouse          19
site                3
studio              1
Name: property_type, dtype: int64

For the categorical features '**area**', '**ber_classification**' and '**property_type**' a mapping approach will be taken to convert these to numerical features going forward.

DATACAMP LINK....................


Convert each categorical variable in turn and update dataframe.

In [544]:
labels = dftrain_adj['area'].astype('category').cat.categories.tolist()

replace_map_comp_1 = {'area' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_map_comp_1)

dftrain_adj.replace(replace_map_comp_1, inplace=True)

{'area': {'Adamstown': 1, 'Ard Na Greine': 2, 'Artane': 3, 'Ashtown': 4, 'Balbriggan': 5, 'Baldoyle': 6, 'Balgriffin': 7, 'Ballinteer': 8, 'Ballsbridge': 9, 'Ballybough': 10, 'Ballyboughal': 11, 'Ballybrack': 12, 'Ballycullen': 13, 'Ballyfermot': 14, 'Ballymun': 15, 'Balrothery': 16, 'Beaumont': 17, 'Blackrock': 18, 'Blanchardstown': 19, 'Bluebell': 20, 'Booterstown': 21, 'Brittas': 22, 'Cabinteely': 23, 'Cabra': 24, 'Carrickmines': 25, 'Castleknock': 26, 'Chapelizod': 27, 'Christchurch': 28, 'Churchtown': 29, 'Citywest': 30, 'Clarehall': 31, 'Clondalkin': 32, 'Clonee': 33, 'Clongriffin': 34, 'Clonshaugh': 35, 'Clonsilla': 36, 'Clonskeagh': 37, 'Clontarf': 38, 'Coolock': 39, 'Crumlin': 40, 'Dalkey': 41, 'Darndale': 42, 'Dartry': 43, 'Donabate': 44, 'Donaghmede': 45, 'Donnybrook': 46, 'Donnycarney': 47, 'Drimnagh': 48, 'Drumcondra': 49, 'Dublin 1': 50, 'Dublin 2': 51, 'Dublin 4': 52, 'Dublin 7': 53, 'Dublin 8': 54, 'Dun Laoghaire': 55, 'Dundrum': 56, 'East Wall': 57, 'Edenmore': 58, 'Fa

In [545]:
labels = dftrain_adj['ber_classification'].astype('category').cat.categories.tolist()

replace_map_comp_2 = {'ber_classification' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_map_comp_2)

dftrain_adj.replace(replace_map_comp_2, inplace=True)

{'ber_classification': {'A1': 1, 'A2': 2, 'A3': 3, 'B1': 4, 'B2': 5, 'B3': 6, 'C1': 7, 'C2': 8, 'C3': 9, 'D1': 10, 'D2': 11, 'E1': 12, 'E2': 13, 'F': 14, 'G': 15, 'SINo666of2006exempt': 16}}


In [546]:
labels = dftrain_adj['property_type'].astype('category').cat.categories.tolist()

replace_map_comp_3 = {'property_type' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_map_comp_3)

dftrain_adj.replace(replace_map_comp_3, inplace=True)

{'property_type': {'apartment': 1, 'bungalow': 2, 'detached': 3, 'duplex': 4, 'end-of-terrace': 5, 'semi-detached': 6, 'site': 7, 'studio': 8, 'terraced': 9, 'townhouse': 10}}


In [547]:
dftrain_adj.head()

,area,bathrooms,beds,ber_classification,latitude,longitude,price,property_type,surface
15,33,3.0,5.0,3,53.400454,-6.445730,935000.0,3,321.0
26,32,4.0,4.0,3,53.316410,-6.385214,485000.0,6,144.0
27,33,3.0,5.0,3,53.401414,-6.446634,935000.0,3,321.0
35,70,5.0,5.0,2,53.375377,-6.056749,1475000.0,3,312.0
38,26,2.0,2.0,3,53.372130,-6.338466,410000.0,1,83.0


In [548]:
dftrain_adj.shape

(1989, 9)

Now all training features are now numeric. There are 9 remaining features (which includes the target 'price').

### Test dataset

The same pre-processing approaches taken on the training dataset will be taken on the test dataset.

In [549]:
# Show first data frame rows.
dftest.head()

,ad_id,area,bathrooms,beds,ber_classification,county,description_block,environment,facility,features,latitude,longitude,no_of_units,property_category,property_type,surface
0,12373510,Skerries,2.0,4.0,G,Dublin,"It's all in the name ""Island View"";. Truly won...",prod,"Parking,Alarm,Oil Fired Central Heating",Breath-taking panoramic views radiate from thi...,53.566881,-6.101148,NaN,sale,bungalow,142.0
1,12422623,Lucan,2.0,3.0,C1,Dublin,REA McDonald - Lucan' s longest established es...,prod,NaN,Gas fired central heating.\nDouble glazed wind...,53.362992,-6.452909,NaN,sale,terraced,114.0
2,12377408,Swords,3.0,4.0,B3,Dublin,REA Grimes are proud to present to the market ...,prod,NaN,Pristine condition throughout\nHighly sought-a...,53.454198,-6.262964,NaN,sale,semi-detached,172.0
3,12420093,Lucan,4.0,3.0,A3,Dublin,"REA McDonald, Lucan' s longest established est...",prod,NaN,A-rated home within a short walk of Lucan Vill...,53.354402,-6.458647,NaN,sale,semi-detached,132.4
4,12417338,Clondalkin,1.0,3.0,E2,Dublin,"Hibernian Auctioneers are delighted to bring, ...",prod,"Parking,Gas Fired Central Heating,Wired for Ca...",Mature Location \nGas Heating \nClose to Ameni...,53.336530,-6.393587,NaN,sale,semi-detached,88.0


In [550]:
# Generate descriptive statistics.
dftest.describe()

,ad_id,bathrooms,beds,latitude,longitude,no_of_units,surface
count,5.000000e+02,500.000000,500.000000,500.000000,500.000000,0.0,500.000000
mean,1.231695e+07,1.994000,2.930000,53.356034,-6.247842,NaN,156.007671
std,1.485832e+05,1.106532,1.191612,0.081905,0.088552,NaN,344.497362
min,1.130615e+07,0.000000,0.000000,53.221348,-6.496987,NaN,33.500000
25%,1.228617e+07,1.000000,2.000000,53.297373,-6.296404,NaN,72.375000
50%,1.237964e+07,2.000000,3.000000,53.339547,-6.243572,NaN,98.000000
75%,1.240544e+07,3.000000,4.000000,53.381650,-6.185055,NaN,138.935000
max,1.242809e+07,8.000000,7.000000,53.619775,-6.064874,NaN,5746.536120


The 'dfcost' data will temporarily be appended to the test dataset to ensure any pre-processing steps effects 'dftest' and 'dfcost' similarly.

In [551]:
dfcost.shape

(500, 2)

Get rid of the ID column as that is already present in the test datset.

In [552]:
dfcost_adj = dfcost.drop(columns='Id')
dfcost_adj.shape
#dfcost_adj

(500, 1)

In [553]:
dftest_adj = pd.concat([dftest, dfcost_adj], axis=1)
dftest_adj.head()

,ad_id,area,bathrooms,beds,ber_classification,county,description_block,environment,facility,features,latitude,longitude,no_of_units,property_category,property_type,surface,Expected
0,12373510,Skerries,2.0,4.0,G,Dublin,"It's all in the name ""Island View"";. Truly won...",prod,"Parking,Alarm,Oil Fired Central Heating",Breath-taking panoramic views radiate from thi...,53.566881,-6.101148,NaN,sale,bungalow,142.0,875000.0
1,12422623,Lucan,2.0,3.0,C1,Dublin,REA McDonald - Lucan' s longest established es...,prod,NaN,Gas fired central heating.\nDouble glazed wind...,53.362992,-6.452909,NaN,sale,terraced,114.0,355000.0
2,12377408,Swords,3.0,4.0,B3,Dublin,REA Grimes are proud to present to the market ...,prod,NaN,Pristine condition throughout\nHighly sought-a...,53.454198,-6.262964,NaN,sale,semi-detached,172.0,440000.0
3,12420093,Lucan,4.0,3.0,A3,Dublin,"REA McDonald, Lucan' s longest established est...",prod,NaN,A-rated home within a short walk of Lucan Vill...,53.354402,-6.458647,NaN,sale,semi-detached,132.4,425000.0
4,12417338,Clondalkin,1.0,3.0,E2,Dublin,"Hibernian Auctioneers are delighted to bring, ...",prod,"Parking,Gas Fired Central Heating,Wired for Ca...",Mature Location \nGas Heating \nClose to Ameni...,53.336530,-6.393587,NaN,sale,semi-detached,88.0,265000.0


Drop columns=['ad_id', 'facility','no_of_units', 'description_block', 'features'] like was done in the training set.

In [554]:
dftest_adj = dftest_adj.drop(columns=['ad_id', 'facility','no_of_units', 'description_block', 'features'])

dftest_adj.shape

(500, 12)

Drop columns=['county', 'environment', 'property_category'] like was done in the training set.

In [555]:
dftest_adj.drop(columns=['county', 'environment', 'property_category'], inplace=True)

dftest_adj.shape

(500, 9)

Drop all rows with NaN present as the sample/observation is incomplete and may be misleading.

In [556]:
dftest_adj.dropna(inplace=True)

dftest_adj.shape

(444, 9)

Remove samples where there are 0 bathrooms and 0 bedrooms.

In [557]:
dftest_adj.drop(dftest_adj[dftest_adj.beds == 0].index, inplace=True)

dftest_adj.drop(dftest_adj[dftest_adj.bathrooms == 0].index, inplace=True)

dftest_adj.shape

(440, 9)

Convert each categorical variable in turn and update dataframe. Use the same mapping as in the training dataset pre-processing.

In [558]:
print(replace_map_comp_1)

dftest_adj.replace(replace_map_comp_1, inplace=True)

{'area': {'Adamstown': 1, 'Ard Na Greine': 2, 'Artane': 3, 'Ashtown': 4, 'Balbriggan': 5, 'Baldoyle': 6, 'Balgriffin': 7, 'Ballinteer': 8, 'Ballsbridge': 9, 'Ballybough': 10, 'Ballyboughal': 11, 'Ballybrack': 12, 'Ballycullen': 13, 'Ballyfermot': 14, 'Ballymun': 15, 'Balrothery': 16, 'Beaumont': 17, 'Blackrock': 18, 'Blanchardstown': 19, 'Bluebell': 20, 'Booterstown': 21, 'Brittas': 22, 'Cabinteely': 23, 'Cabra': 24, 'Carrickmines': 25, 'Castleknock': 26, 'Chapelizod': 27, 'Christchurch': 28, 'Churchtown': 29, 'Citywest': 30, 'Clarehall': 31, 'Clondalkin': 32, 'Clonee': 33, 'Clongriffin': 34, 'Clonshaugh': 35, 'Clonsilla': 36, 'Clonskeagh': 37, 'Clontarf': 38, 'Coolock': 39, 'Crumlin': 40, 'Dalkey': 41, 'Darndale': 42, 'Dartry': 43, 'Donabate': 44, 'Donaghmede': 45, 'Donnybrook': 46, 'Donnycarney': 47, 'Drimnagh': 48, 'Drumcondra': 49, 'Dublin 1': 50, 'Dublin 2': 51, 'Dublin 4': 52, 'Dublin 7': 53, 'Dublin 8': 54, 'Dun Laoghaire': 55, 'Dundrum': 56, 'East Wall': 57, 'Edenmore': 58, 'Fa

In [559]:
print(replace_map_comp_2)

dftest_adj.replace(replace_map_comp_2, inplace=True)

{'ber_classification': {'A1': 1, 'A2': 2, 'A3': 3, 'B1': 4, 'B2': 5, 'B3': 6, 'C1': 7, 'C2': 8, 'C3': 9, 'D1': 10, 'D2': 11, 'E1': 12, 'E2': 13, 'F': 14, 'G': 15, 'SINo666of2006exempt': 16}}


In [560]:
print(replace_map_comp_3)

dftest_adj.replace(replace_map_comp_3, inplace=True)

{'property_type': {'apartment': 1, 'bungalow': 2, 'detached': 3, 'duplex': 4, 'end-of-terrace': 5, 'semi-detached': 6, 'site': 7, 'studio': 8, 'terraced': 9, 'townhouse': 10}}


In [561]:
dftest_adj.head()

,area,bathrooms,beds,ber_classification,latitude,longitude,property_type,surface,Expected
0,128,2.0,4.0,15,53.566881,-6.101148,2,142.0,875000.0
1,90,2.0,3.0,7,53.362992,-6.452909,9,114.0,355000.0
2,134,3.0,4.0,6,53.454198,-6.262964,6,172.0,440000.0
3,90,4.0,3.0,3,53.354402,-6.458647,6,132.4,425000.0
4,32,1.0,3.0,13,53.336530,-6.393587,6,88.0,265000.0


In [562]:
dftest_adj.shape

(440, 9)

Now all test features are now numeric. There are 9 remaining features (which includes the target 'price').

### Expected Cost dataset

In [565]:
cost_data = [dftest_adj['Expected']]

header_name = ['Expected']

dfcost_updated = pd.concat(cost_data, axis=1, keys=header_name)

dftest_adj.drop(columns='Expected', inplace=True)

dfcost_updated.shape

(440, 1)

In [566]:
dftest_adj.shape

(440, 8)

In [567]:
# Show first data frame rows.
dfcost_updated.head()

,Expected
0,875000.0
1,355000.0
2,440000.0
3,425000.0
4,265000.0


In [568]:
# Generate descriptive statistics.
dfcost_updated.describe()

,Expected
count,4.400000e+02
mean,5.431734e+05
std,4.711011e+05
min,1.250000e+05
25%,2.950000e+05
50%,4.175000e+05
75%,5.950000e+05
max,5.250000e+06


# PIECEWISE REGRESSION

## Full Model

In [ ]:
# select some features columns just for the baseline model
# assume not all of the features are informative or useful
# in this exercise you could try all of them if possible

featrain = ['feature_1','feature_2','feature_3','cost']
# dropna: remove missing values
df_subset_train = dftrain[featrain].dropna(axis=0)

featest = ['feature_1','feature_2','feature_3']
df_subset_test  =  dftest[featest].dropna(axis=0)

# cost
df_cost = df_cost[df_cost.index.isin(df_subset_test.index)]

In [ ]:
# model
with pm.Model() as model:
    #prior over the parameters of linear regression
    alpha = pm.Normal('alpha', mu=0, sigma=30)
    #we have one beta for each column of Xn
    beta = pm.Normal('beta', mu=0, sigma=30, shape=Xn_train.shape[1])
    #prior over the variance of the noise
    sigma = pm.HalfCauchy('sigma_n', 5)
    #linear regression model in matrix form
    mu = alpha + pm.math.dot(beta, Xn_train.T)
    #likelihood, be sure that observed is a 1d vector
    like = pm.Normal('like', mu=mu, sigma=sigma, observed=yn_train[:,0])
    

In [ ]:
# prediction
ll=np.mean(posterior['alpha']) + np.dot(np.mean(posterior['beta'],axis=0), Xn_test.T)
y_pred_BLR = np.exp(yscaler.inverse_transform(ll.reshape(-1,1)))[:,0]
print("MAE = ",(np.mean(abs(y_pred_BLR - y_test))))
print("MAPE = ",(np.mean(abs(y_pred_BLR - y_test) / y_test)))

## Clustering

### Full Model

In [ ]:
# training gaussian mixture model 
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4)


### Clusters

In [ ]:
# train clusters



In [ ]:
# test clusters


## Piecewise Model

In [ ]:
# model_0
with pm.Model() as model_0:
  # prior over the parameters of linear regression
  alpha = pm.Normal('alpha', mu=0, sigma=30)
  # we have a beta for each column of Xn0
  beta = pm.Normal('beta', mu=0, sigma=30, shape=Xn0.shape[1])
  # prior over the variance of the noise
  sigma = pm.HalfCauchy('sigma_n', 5)
  # linear regression relationship
  #linear regression model in matrix form
  mu = alpha + pm.math.dot(beta, Xn0.T)
  # likelihood, be sure that observed is a 1d vector
  like = pm.Normal('like', mu=mu, sigma=sigma, observed=yn0[:,0])



##Simulations

### Only Cluster 0

## Overall

## Test set performance

### PPC on the Test set



# SUMMARY


https://www.datacamp.com/community/tutorials/categorical-data#encoding 